# 네이버 매장정보 크롤링

#### 자료 
https://www.localdata.go.kr/devcenter/dataDown.do?menuNo=20001

1. 관광식당, 일반음식점, 휴게음식점 데이터셋 다운로드(07_24_01_P_.csv.zip, 07_24_04_P_.csv.zip , 07_24_05_P_.csv.zip)
2. 다운로드한 파일들을 data 폴더에 넣어주세요. (압축풀지말고)

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [24]:
! pip install --upgrade selenium webdriver_manager beautifulsoup4 numpy pandas tqdm chromedriver_autoinstaller webbrowser


  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pandas-2.2.0-cp310-cp310-win_amd64.whl.metadata (19 kB)


ERROR: Could not find a version that satisfies the requirement webbrowser (from versions: none)
ERROR: No matching distribution found for webbrowser


In [25]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 

import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os


from selenium import webdriver
import chromedriver_autoinstaller
import webbrowser

In [26]:
df = pd.read_csv('./data/filtered_data_서울특별시강남구.csv')
df.head(5)

,영업상태명,소재지전체주소,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),위생업태명,시도,지역구
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,2023-09-22 11:33:55,2023-09-24 00:17:57,한식,202203.971845,444853.761458,한식,서울특별시,강남구
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,2023-09-22 10:20:03,2023-09-24 00:17:57,한식,202462.794327,443827.961945,한식,서울특별시,강남구
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,2023-12-01 17:58:46,2023-12-03 00:17:36,기타,205130.591679,445590.096838,기타,서울특별시,강남구
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,2023-12-01 17:01:54,2023-12-03 00:17:36,한식,203470.848439,447369.579852,한식,서울특별시,강남구
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,2023-12-01 16:01:58,2023-12-03 00:17:36,일식,204447.401189,445478.066611,일식,서울특별시,강남구


In [27]:
df.isna().sum()

영업상태명       0
소재지전체주소     0
도로명전체주소    95
사업장명        0
최종수정시점      0
데이터갱신일자     0
업태구분명       1
좌표정보(x)    81
좌표정보(y)    81
위생업태명       1
시도          0
지역구         0
dtype: int64

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16384 entries, 0 to 16383
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    16384 non-null  object 
 1   소재지전체주소  16384 non-null  object 
 2   도로명전체주소  16289 non-null  object 
 3   사업장명     16384 non-null  object 
 4   최종수정시점   16384 non-null  object 
 5   데이터갱신일자  16384 non-null  object 
 6   업태구분명    16383 non-null  object 
 7   좌표정보(x)  16303 non-null  float64
 8   좌표정보(y)  16303 non-null  float64
 9   위생업태명    16383 non-null  object 
 10  시도       16384 non-null  object 
 11  지역구      16384 non-null  object 
dtypes: float64(2), object(10)
memory usage: 1.5+ MB


In [29]:
# 업태구분명 확인
unique_values = df['업태구분명'].unique()
unique_values

# 필요없는 카테고리(카페, 술, 출장요리 등등) 제거
제외 = ['전통찻집','호프/통닭','뷔페식','출장조리','정종/대포집/소주방','이동조리', '감성주점','까페','라이브카페','키즈카페','커피숍','편의점', '일반조리판매','아이스크림', '떡카페', '철도역구내', '푸드트럭', '과자점', '다방', '관광호텔']
for i in 제외 :
    df = df.drop(df[df['업태구분명'] == i].index)

# 필요없는 칼럼 제거 
df.drop(['최종수정시점','데이터갱신일자','위생업태명'],axis=1,inplace=True)

# 도로명 주소만 추출
df['상세주소'] = df['도로명전체주소'].apply(lambda x: x.split(' ',4)[2] if pd.notna(x) else None) + df['도로명전체주소'].apply(lambda x: x.split(' ',4)[3] if pd.notna(x) else None)
df.reset_index(inplace=True, drop=True)

df.head(21)

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,"
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,"
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,기타,205130.591679,445590.096838,서울특별시,강남구,"영동대로513,"
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,"
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,일식,204447.401189,445478.066611,서울특별시,강남구,"봉은사로72길11,"
5,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로204,"
6,영업/정상,서울특별시 강남구 삼성동 124-27,"서울특별시 강남구 테헤란로77길 41, 지상1층 (삼성동)",마스터즈,한식,204648.781835,445194.685321,서울특별시,강남구,"테헤란로77길41,"
7,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길11,"
8,영업/정상,서울특별시 강남구 대치동 907-12 아티스톤,"서울특별시 강남구 역삼로 409, 지상1층 102호 (대치동, 아티스톤)",분식을 품다,분식,204506.415174,444321.238711,서울특별시,강남구,"역삼로409,"
9,영업/정상,서울특별시 강남구 역삼동 660-25,"서울특별시 강남구 언주로103길 35, 지하1층 101호 (역삼동)",곰푸드,한식,203218.312570,444859.547190,서울특별시,강남구,"언주로103길35,"


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12535 entries, 0 to 12534
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    12535 non-null  object 
 1   소재지전체주소  12535 non-null  object 
 2   도로명전체주소  12475 non-null  object 
 3   사업장명     12535 non-null  object 
 4   업태구분명    12534 non-null  object 
 5   좌표정보(x)  12490 non-null  float64
 6   좌표정보(y)  12490 non-null  float64
 7   시도       12535 non-null  object 
 8   지역구      12535 non-null  object 
 9   상세주소     12475 non-null  object 
dtypes: float64(2), object(8)
memory usage: 979.4+ KB


# 셀레니움

- 원래 코드로 하면 계속 오류났음 

    -> ChromeDriverManager로 하면 오류나는 듯 해서 chromedriver_autoinstaller로 했습니다

In [31]:
# 샘플용 데이터 200개 추출 

# # 1. 인덱스 기준 200개 추출
# df_200 = df.iloc[:200,:]

# 2. 랜덤으로 100개 추출
df_200 = df.sample(n=300, random_state=42)
df_200.reset_index(inplace=True, drop=True)
df_200.head(10)

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소
0,영업/정상,서울특별시 강남구 역삼동 817-18 정해빌딩 2층동,"서울특별시 강남구 강남대로96길 15, 2층동 (역삼동,정해빌딩)",육미당 몽산,경양식,202427.491136,444209.275456,서울특별시,강남구,"강남대로96길15,"
1,영업/정상,서울특별시 강남구 논현동 98-6,"서울특별시 강남구 언주로148길 6, 지하1층 (논현동)",케렌시아.장금이네,한식,203034.658806,446445.535888,서울특별시,강남구,"언주로148길6,"
2,영업/정상,서울특별시 강남구 신사동 620-0번지 4층407호,"서울특별시 강남구 압구정로34길 11 (신사동,4층407호)",로데오,경양식,202666.717098,447239.218593,서울특별시,강남구,압구정로34길11
3,영업/정상,서울특별시 강남구 청담동 90-3번지 지하1층,"서울특별시 강남구 도산대로61길 27 (청담동,지하1층)",2,경양식,203743.205741,447049.623113,서울특별시,강남구,도산대로61길27
4,영업/정상,서울특별시 강남구 논현동 192-1 지하1층,"서울특별시 강남구 논현로113길 7, 지하1층 B101호 (논현동, 리더스빌)",참달달한포차,한식,202805.403854,445111.004762,서울특별시,강남구,"논현로113길7,"
5,영업/정상,서울특별시 강남구 논현동 85-10,"서울특별시 강남구 언주로133길 15, 지상1층 (논현동)",작은뜰,경양식,202945.074532,445978.077904,서울특별시,강남구,"언주로133길15,"
6,영업/정상,서울특별시 강남구 논현동 111-5,"서울특별시 강남구 선릉로135길 6, 지상2층 (논현동)",스타일비건,경양식,203471.666260,446283.746446,서울특별시,강남구,"선릉로135길6,"
7,영업/정상,서울특별시 강남구 신사동 656-2,"서울특별시 강남구 선릉로155길 25, 1층 103호 (신사동)",수리다라,한식,203244.739677,447040.262357,서울특별시,강남구,"선릉로155길25,"
8,영업/정상,서울특별시 강남구 논현동 11-5번지 지상2층,"서울특별시 강남구 논현로149길 51, 지상2층 (논현동)",홀리즌,패스트푸드,202100.731049,446102.757158,서울특별시,강남구,"논현로149길51,"
9,영업/정상,서울특별시 강남구 청담동 34-9번지,서울특별시 강남구 학동로67길 12 (청담동),발리밥,경양식,204063.240151,446315.452315,서울특별시,강남구,학동로67길12


In [32]:
! pip install webdriver-manager

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)


In [34]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정

# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()


# # 주소 이동
# url = 'https://map.naver.com/'
# driver.get(url)
# time.sleep(1)

엑셀 파일엔 정보가 존재하는데 실제 식당이 없는경우 or 서브웨이처럼 이름은 같은데 지점만 다른 경우를 대비해서, '도로명 주소 + 식당이름' 으로 크롤링함
 

In [35]:
# 상세주소 + 사업장명으로 검색하는 코드 
# 식당이 존재하지 않는경우 검색결과가 없는데, 검색여부를 보고 실제 존재하는 식당만 추릴것임.

# 칼럼 생성 
df_200['검색여부'] = ''

# tqdm 데코레이터를 사용하여 반복문 진행 상황을 표시
for i in tqdm(range(len(df_200.loc[:,['사업장명','상세주소']]))):

    global driver  # 전역 변수로 driver 사용

    name = df_200['상세주소'][i] + ' ' + df_200['사업장명'][i]

    # 매장이 검색되는 경우 
    try :
        searchIframe = driver.find_element(By.ID,'searchIframe')
        driver.switch_to.frame(searchIframe)
        df_200['검색여부'][i] = '가능'

    # 매장이 검색되지 않는 경우 
    except :
        # 전처리 ((주)빼기, 첫번째 단어만 검색 ex. '언주로 34,서브웨이 도곡본점' -> '언주로 34,서브웨이') 
        new_name = df_200['사업장명'][i].replace('(주)', '')
        new_name = new_name.split(' ', 2)[:2]
        new_name = ' '.join(new_name)
        name = df_200['상세주소'][i] + ' ' + new_name

        driver.get('https://map.naver.com/p/search/{}'.format(name))
        time.sleep(2.2)  

        try :
            searchIframe = driver.find_element(By.ID,'searchIframe')
            driver.switch_to.frame(searchIframe)
            df_200['검색여부'][i] = '가능'
            df_200['사업장명'][i] = new_name # 사업장명 이름도 바꿔놓기
            
        except NoSuchElementException:
            continue



  0%|          | 1/300 [00:03<18:00,  3.61s/it]

강남대로96길15, 육미당 몽산 해결


  2%|▏         | 5/300 [00:14<13:44,  2.79s/it]

논현로113길7, 참달달한포차 해결


  2%|▏         | 6/300 [00:17<13:10,  2.69s/it]

...

논현로150길16, 전주식당 해결


 94%|█████████▎| 281/300 [12:11<00:48,  2.57s/it]

테헤란로322, 아임샌드 해결


 94%|█████████▍| 282/300 [12:14<00:46,  2.60s/it]


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [1]:
# 저장 
df_200.to_excel('data/df_200_test.xlsx', index=False)


NameError: name 'df_200' is not defined

In [ ]:
# 기존

# for i in range(0,len(df_200.loc[:,['사업장명','상세주소']])):
#     # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
#     name = df_200['상세주소'][i] + ' ' +df_200['사업장명'][i]
#     driver.get('https://map.naver.com/p/search/{}'.format(name))

#     time.sleep(3)  

#     try :
#         searchIframe = driver.find_element(By.ID,'searchIframe')
#         driver.switch_to.frame(searchIframe)
#         time.sleep(3) 
#         df_200['검색여부'][i] = '가능'
#         print(name,'검색 완')
#     except NoSuchElementException:
#         print("식당 존재 x ", name)
#         df_200['검색여부'][i] = '불가능'
#         continue
    
# # 검색 가능/불가능 추가 확인 
# df_200

# 문제 1

식당 존재 x  압구정로343, 메밀집 압구정갤러리아점 -> 메밀집 갤러리아백화점 명품관 WEST점

식당 존재 x  언주로123, (주)일일향 도곡1호점 -> 언주로 123 일일향 도곡점

식당 존재 x  강남대로152길64, 이끼롤까스 가로수길본점 -> 서울 강남구 강남대로156길 35 1층

# 문제 2 
뻑남


실제 존재하는 식당만 추려냄 


In [ ]:
new_df_200 = df_200.loc[df_200['검색여부']=='가능',:]
new_df_200
count_possible_stores = len(new_df_200)

print(f'\'가능\'한 매장 수: {count_possible_stores}')

카테고리 + 이름 없으면 ㄱㄱ 

In [ ]:
new_df_200.to_csv('data/new_df_200.csv')